# Introduction to Code Rewriter using Scalpel



The code rewriter module is designed as a fundamental function for supporting systematic changes of existing Python programs. Two preliminary usages of this function are to (1) simplify the programs for better static analysis and (2) optimize or repair problematic programs.

## Let's install Scalpel first

Use the command in your virtual environment to install Scalpel.
```console
pip install python-scalpel
```

We will now import all the necessary module, including Rewriter from Scalpel.

In [1]:
import ast
import os
import sys
import astor
from scalpel.rewriter import Rewriter

Now, we provide the code we want to rewrite and also define the expected output for the rewriter module.

In [2]:
src = """
a = list()
b = dict()
"""
expected_src = """
a = []
b = {}
"""

Lets, define the function to establish rules for code rewriting. The function below represents the rewriting rules for keyword based initialization to character based initialization of lists and dictionary objects.

In [3]:
def rewrite_rules(node) -> list:
    # This function takes an AST node and returns a list of new AST nodes
    # that represent the rewritten code.
    if isinstance(node, ast.Assign):
        if isinstance(node.value, ast.Call) and hasattr(node.value.func, "id"):
            if node.value.func.id == "list":
                new_assign_value = ast.List(elts=[], ctx=ast.Load())
                new_stmt = ast.Assign(node.targets, new_assign_value)
                return [new_stmt]
            
            if node.value.func.id == "dict":
                new_assign_value = ast.Dict(keys=[], values=[])
                new_stmt = ast.Assign(node.targets, new_assign_value)
                return [new_stmt]

Finally, to rewrite the code using **Rewriter** , we have to initialize the rewriter object and them pass the source code and rewrite rules as parameters.

In [4]:
rewriter = Rewriter(src)
new_src = rewriter.rewrite( src, rule_func=rewrite_rules)
print(new_src)

a = []
b = {}



#### Here is an another example for code rewriting using Scalpel for renaming variables in source code.

In [5]:
src_var_rename = """
def func(a,b):
    c = a+b
    return c
"""

target_var_rename = """
def func(x,y):
    z = x+y
    return z
"""

This rewriting rule renames variable(s) with name **z** to **c**. This is done by checking all the nodes in the AST and replacing all the instance of variable **z** to **c** in each of the node.

In [6]:
def rewrite_rules_var_rename(node) -> list:
    old_name = "c"
    new_name = "z"
    for tmp_node in ast.walk(node):
        if isinstance(tmp_node, ast.Name):
            if tmp_node.id == old_name:
                tmp_node.id = new_name
    return [node]

Finally, we can then re-write the source code with above defined rules using rewriter module.

In [7]:
rewriter_var_rename = Rewriter(src_var_rename)
new_src_var_rename = rewriter_var_rename.rewrite(src_var_rename, rule_func=rewrite_rules_var_rename)
print(new_src_var_rename)

def func(a, b):
    z = a + b
    return z

